In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

In [19]:
torch.cuda.is_available()

False

In [21]:
torch.cuda.device()

TypeError: __init__() missing 1 required positional argument: 'device'

In [4]:
spy_df = pd.read_csv("../data/spy_data.csv")

In [5]:
spy_df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
0,2003-07-22 00:00:00-04:00,67.246893,67.737498,66.722215,67.573959,49968300,0.0,0.0,0.0
1,2003-07-23 00:00:00-04:00,67.601233,67.764766,66.967535,67.621674,37275400,0.0,0.0,0.0
2,2003-07-24 00:00:00-04:00,68.132719,68.371207,67.028862,67.110626,40896200,0.0,0.0,0.0
3,2003-07-25 00:00:00-04:00,67.226474,68.337147,66.804007,68.296265,43241100,0.0,0.0,0.0
4,2003-07-28 00:00:00-04:00,68.391634,68.807285,67.914654,68.044121,34382800,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
5030,2023-07-17 00:00:00-04:00,449.130005,451.929993,449.079987,450.839996,52680200,0.0,0.0,0.0
5031,2023-07-18 00:00:00-04:00,450.500000,454.859985,450.049988,454.190002,80744400,0.0,0.0,0.0
5032,2023-07-19 00:00:00-04:00,455.010010,456.429993,454.109985,455.200012,65891700,0.0,0.0,0.0
5033,2023-07-20 00:00:00-04:00,454.170013,455.100006,451.440002,452.179993,70591600,0.0,0.0,0.0


In [7]:
 df = spy_df
timeseries = df[["Close"]].values.astype('float32')

In [8]:
# train-test split for time series
train_size = int(len(timeseries) * 0.67)
test_size = len(timeseries) - train_size
train, test = timeseries[:train_size], timeseries[train_size:]

In [9]:
timeseries

array([[ 67.57396],
       [ 67.62167],
       [ 67.11063],
       ...,
       [455.2    ],
       [452.18   ],
       [452.18   ]], dtype=float32)

In [10]:
def create_dataset(dataset, lookback):
    """Transform a time series into a prediction dataset
    
    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
    """
    X, y = [], []
    for i in range(len(dataset)-lookback):
        feature = dataset[i:i+lookback]
        target = dataset[i+1:i+lookback+1]
        X.append(feature)
        y.append(target)
    return torch.tensor(X), torch.tensor(y)

In [11]:
lookback = 4
X_train, y_train = create_dataset(train, lookback=lookback)
X_test, y_test = create_dataset(test, lookback=lookback)

C:\Users\gabes\AppData\Local\Temp\ipykernel_12420\778121417.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:248.)
  return torch.tensor(X), torch.tensor(y)


In [13]:
class AirModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50, 1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x

In [14]:
model = AirModel()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=8)

In [15]:
n_epochs = 2000
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    if epoch % 100 != 0:
        continue
    model.eval()
    with torch.no_grad():
        y_pred = model(X_train)
        train_rmse = np.sqrt(loss_fn(y_pred, y_train))
        y_pred = model(X_test)
        test_rmse = np.sqrt(loss_fn(y_pred, y_test))
    print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))

Epoch 0: train RMSE 96.5770, test RMSE 304.6360


KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    # shift train predictions for plotting
    train_plot = np.ones_like(timeseries) * np.nan
    y_pred = model(X_train)
    y_pred = y_pred[:, -1, :]
    train_plot[lookback:train_size] = model(X_train)[:, -1, :]
    # shift test predictions for plotting
    test_plot = np.ones_like(timeseries) * np.nan
    test_plot[train_size+lookback:len(timeseries)] = model(X_test)[:, -1, :]

In [ ]:
# plot
plt.plot(timeseries)
plt.plot(train_plot, c='r')
plt.plot(test_plot, c='g')
plt.show()